In [42]:
# import external modules
import sys, os

ROOT_PATH = os.path.abspath(".").split("src")[0]
module_path = os.path.abspath(os.path.join(ROOT_PATH + "src/utils/"))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rc 
from datetime import datetime

import functions as f
import dl_functions as dlf

from cognite.client import CogniteClient
client = CogniteClient(api_key=os.environ['COGNITE_API_SECRET'])

In [43]:
# set plot settings
sns.set()
sns.set_context('paper')
sns.set_style('whitegrid', {'axes.grid': True, 'grid.linestyle': '--'})

rc('figure', figsize=(12,6), dpi=200)
rc('xtick', labelsize=12)
rc('ytick', labelsize=12)
rc('axes', labelsize=13, titlesize=14)
rc('legend', fontsize=14, handlelength=2)
rc('font', family='serif')
rc('text', color="#000000")
rc('xtick', color="#000000")
rc('ytick', color="#000000")

In [44]:
df_train, df_valid, df_test = f.load_data()
#df_train, df_valid, df_test = f.load_data(dummy_data=True, dummy_obs=5000)
scaling_stats, ts, ts_train, ts_valid, ts_test = f.load_metadata()
timestamps = np.load(ROOT_PATH + "data/metadata/timestamps/timestamps.npy")

# split datasets into features and targets
x_train, y_train = f.split_dataset(df_train.values, delay=1)
x_valid, y_valid = f.split_dataset(df_valid.values, delay=1)
x_test, y_test = f.split_dataset(df_test.values, delay=1)

# metadata
target_tags = df_train.columns.values[:3]
feature_tags = df_train.columns.values[3:]
target_stds = stats.loc[target_tags,"Std"].values

In [45]:
START = datetime(2017,12, 24)  ## the start date of the asset time series interval
END = datetime(2018, 2, 24)    ## the end date of the asset time series interval 
TIME_INTERVAL_MINUTES = 1      ## the minutes between each reading in the time series
GRANULARITY = str(TIME_INTERVAL_MINUTES) + 'm' ## the string representation of TIME_INTERVAL_MINUTES
AGGREGATES = ['average']       ## the aggregates to extract from the Cognite API

def read_data(tags, start=START, end=END, granularity=GRANULARITY, aggregates=AGGREGATES):
    """
    Extract time series from the Cognite API given a list of tags.
    
    Args: 
        tags ([String]): A list of tags to extract time series of.
        start (datetime): A datetime object specifying the start of the interval
        end (datetime): A datetime object specifying the end of the interval
        granularity (String): Time between two samplings
        aggregates ([String]): Data properties to extract
    
    Returns: 
        pd.DataFrame: The dataframe of tags data
    """
    
    data = client.datapoints.get_datapoints_frame(list(tags), 
                                                  start=start, 
                                                  end=end, 
                                                  granularity=granularity, 
                                                  aggregates=aggregates)
    
    columns = [name.split("|")[0] for name in data.columns]
    data.columns=columns
    
    return data

def load_predictor_data(timestamps, tags=feature_tags):
    """
    Load data for the predictor tags. 
    Cognite API has limit for 100 assets at a time, so must create fetch intervals.
    """
    data_predictors = pd.DataFrame(timestamps, columns=["timestamp"])
    intervals = np.arange(0,len(tags),100)

    if intervals[-1] != len(tags):
        intervals = np.append(intervals, len(tags))

    for i, j in enumerate(intervals[1:]):
        from_int = intervals[i]
        to_int = j

        tmp_tags = list(tags[from_int:to_int])
        tmp_data = read_data(tmp_tags)      

        data_predictors = data_predictors.join(tmp_data.set_index("timestamp"), on="timestamp")
        print("-- Completed tags {0} to {1}".format(from_int, to_int))
    
    return data_predictors

In [61]:
data_targets = read_data(target_tags)
data_features = load_predictor_data(data_targets.timestamp, feature_tags)

data_targets.index = pd.to_datetime(data_targets.timestamp, unit="ms")
data_features.index = pd.to_datetime(data_features.timestamp,unit="ms")
data_targets = data_targets.iloc[:,1:]
data_features = data_features.iloc[:,1:]

data_complete = pd.concat([data_targets, data_features], axis=1)

-- Completed tags 0 to 6


In [72]:
stats = f.get_stats_properties(data_complete)
stats

,Mean,Median,Std,Max,Min,1st Qu.,3rd Qu.,NAs
VAL_23-FT-92537-01:X.Value,161417.157637,161285.910090,3451.291254,224396.129479,147442.173910,159105.709228,163550.644406,18.0
VAL_23-TT-92539:X.Value,125.079347,125.064337,1.183979,130.570725,119.512739,124.280447,125.882580,1227.0
VAL_23-PT-92539:X.Value,12.698143,12.702663,0.245375,14.656747,11.752882,12.518977,12.875018,21.0
VAL_23_ZT_92543:Z.X.Value,36.378872,36.429495,4.231451,60.949738,16.510971,33.275271,39.481204,1006.0
VAL_23-PT-92523:X.Value,2.899429,2.891025,0.078651,4.021907,2.605437,2.842954,2.947648,31.0
VAL_23-PDT-92534:X.Value,103.192175,102.978120,4.078539,157.089903,85.976939,100.383456,105.728059,20.0
VAL_23_TT_92532:Z.X.Value,33.367604,33.441364,1.174527,39.998225,27.942626,32.547140,34.193537,960.0
VAL_23-TIC-92504:Z.X.Value,39.653444,39.734615,1.341746,45.121540,35.067940,38.832164,40.354978,50.0
VAL_23_KA_9101_M01_62C:Z.X.Value,9026.323034,9026.773560,163.287563,11349.886165,8230.075978,8917.354690,9134.357653,28.0


In [100]:
sparsity_per_feature = 100 * stats['NAs'] / len(data_complete)
avg_sparsity = np.average(sparsity_per_feature)
total_sparsity = sum(sparsity_per_feature)

print("Sparsity per feature\n", sparsity_per_feature)
print("Avg. sparsity (%):", avg_sparsity)
print("Total sparsity:", total_sparsity)

print(sum(stats['NAs']))
print(len(data_complete))

Sparsity per feature
 VAL_23-FT-92537-01:X.Value          0.020297
VAL_23-TT-92539:X.Value             1.383549
VAL_23-PT-92539:X.Value             0.023679
VAL_23_ZT_92543:Z.X.Value           1.134352
VAL_23-PT-92523:X.Value             0.034955
VAL_23-PDT-92534:X.Value            0.022552
VAL_23_TT_92532:Z.X.Value           1.082483
VAL_23-TIC-92504:Z.X.Value          0.056379
VAL_23_KA_9101_M01_62C:Z.X.Value    0.031572
Name: NAs, dtype: float64
Avg. sparsity (%): 0.4210908771995765
Total sparsity: 3.7898178947961885
3361.0
88685


In [102]:
## If the value exists before and after, then impute it linearly
data_imp = data_complete.copy()

for col in range(data_imp.shape[-1]):
    idxs = np.where(data_imp.iloc[:,col].isna())[0]
    for idx in idxs: 
        
        # if the index is the first or last, continue
        if idx == 0 or idx == len(data_complete):
            continue
        
        curr_val = data_imp.iloc[idx, col]
        next_val = data_imp.iloc[idx+1, col]
        last_val = data_imp.iloc[idx-1, col]
        
        # if value is 
        if np.isnan(curr_val) and not np.isnan(last_val) and not np.isnan(next_val):
            data_imp.iloc[idx,col] = np.average([last_val, next_val])

new_percentage = round(100*np.average(data_imp.isna().sum().values/len(data_imp)),3)
old_percentage = round(100*np.average(data_complete.isna().sum().values/len(data_complete)),3)
print("Old sparsity: {}%, ".format(old_percentage) + 
      "New sparsity: {}%, ".format(new_percentage) + 
      "Improvement: {}%".format(round(100*(1-new_percentage/old_percentage),3)))

get_stats_properties(data=data_imp)

Old sparsity: 0.421%, New sparsity: 0.069%, Improvement: 83.61%


,Mean,Median,Std,Max,Min,1st Qu.,3rd Qu.,NAs
VAL_23-FT-92537-01:X.Value,161417.151479,161285.897240,3451.246477,224396.129479,147442.173910,159105.709228,163550.644406,14.0
VAL_23-TT-92539:X.Value,125.080403,125.066368,1.183835,130.570725,119.512739,124.282265,125.883848,105.0
VAL_23-PT-92539:X.Value,12.698141,12.702644,0.245368,14.656747,11.752882,12.518985,12.875013,12.0
VAL_23_ZT_92543:Z.X.Value,36.381522,36.433169,4.228360,60.949738,16.510971,33.276135,39.487660,221.0
VAL_23-PT-92523:X.Value,2.899429,2.891026,0.078650,4.021907,2.605437,2.842954,2.947647,27.0
VAL_23-PDT-92534:X.Value,103.192164,102.978055,4.078517,157.089903,85.976939,100.383460,105.728014,19.0
VAL_23_TT_92532:Z.X.Value,33.365640,33.439919,1.175940,39.998225,27.942626,32.545527,34.192363,92.0
VAL_23-TIC-92504:Z.X.Value,39.653399,39.734609,1.341737,45.121540,35.067940,38.832049,40.354917,42.0
VAL_23_KA_9101_M01_62C:Z.X.Value,9026.320025,9026.769848,163.286089,11349.886165,8230.075978,8917.354690,9134.357653,20.0


In [103]:
sum(get_stats_properties(data=data_imp)["NAs"])

552.0

In [109]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [110]:
%%R -i data_imp -w 5 -h 5 --units in -r 200

# install.packages(c("Amelia", "stringr"))  ## uncomment if you need to install the libraries
library(Amelia)
library(stringr)

print(sum(is.na(data_imp)))
amelia.data <- amelia(data_imp, m = 1, parallel = "multicore")

imput <- 1
am.data <- amelia.data$imputations[[imput]]
print(sum(is.na(am.data)))
percentage_na <- round(sum(is.na(am.data)) / nrow(am.data),2)
cat(str_glue('Percentage NAs: {percentage_na}%'), '\n')

# write to file
write.table(
    am.data, 
    "../../data/amelia_data.csv",
    row.names = FALSE,
    sep = ","
)

[1] 552
-- Imputation 1 --

  1  2

[1] 0
Percentage NAs: 0% 


In [111]:
# read the data
path=ROOT_PATH+"data/amelia_data.csv"
data_amelia = pd.read_csv(path, sep=",")

data_amelia.columns = data_imp.columns

# if not all columns have 0 NA, use forward filling
if not np.all(data_amelia.isna().sum() == 0):
    data_amelia = data_amelia.fillna(method="ffill")
    data_amelia = data_amelia.fillna(method="backfill")

assert np.all(data_amelia.isna().sum() == 0)

# delete the data file
os.remove(path)

In [113]:
data_amelia.isna().sum()

VAL_23-FT-92537-01:X.Value          0
VAL_23-TT-92539:X.Value             0
VAL_23-PT-92539:X.Value             0
VAL_23_ZT_92543:Z.X.Value           0
VAL_23-PT-92523:X.Value             0
VAL_23-PDT-92534:X.Value            0
VAL_23_TT_92532:Z.X.Value           0
VAL_23-TIC-92504:Z.X.Value          0
VAL_23_KA_9101_M01_62C:Z.X.Value    0
dtype: int64

In [116]:
print(data_imp.isna().sum())
data_filled = data_imp.fillna(method='ffill')
data_filled = data_filled.fillna(method='backfill')
print(data_filled.isna().sum())

VAL_23-FT-92537-01:X.Value           14
VAL_23-TT-92539:X.Value             105
VAL_23-PT-92539:X.Value              12
VAL_23_ZT_92543:Z.X.Value           221
VAL_23-PT-92523:X.Value              27
VAL_23-PDT-92534:X.Value             19
VAL_23_TT_92532:Z.X.Value            92
VAL_23-TIC-92504:Z.X.Value           42
VAL_23_KA_9101_M01_62C:Z.X.Value     20
dtype: int64
VAL_23-FT-92537-01:X.Value          0
VAL_23-TT-92539:X.Value             0
VAL_23-PT-92539:X.Value             0
VAL_23_ZT_92543:Z.X.Value           0
VAL_23-PT-92523:X.Value             0
VAL_23-PDT-92534:X.Value            0
VAL_23_TT_92532:Z.X.Value           0
VAL_23-TIC-92504:Z.X.Value          0
VAL_23_KA_9101_M01_62C:Z.X.Value    0
dtype: int64


,Mean,Median,Std,Max,Min,1st Qu.,3rd Qu.,NAs
VAL_23-FT-92537-01:X.Value,0.999999,0.999997,0.999967,1.0,1.000000,0.999997,1.000000,NaN
VAL_23-TT-92539:X.Value,1.000000,1.000003,1.000522,1.0,0.993599,0.999998,1.000002,NaN
VAL_23-PT-92539:X.Value,1.000001,1.000001,1.000049,1.0,1.000000,1.000001,1.000003,NaN
VAL_23_ZT_92543:Z.X.Value,0.999938,0.999900,1.000024,1.0,1.000000,0.999987,0.999923,NaN
VAL_23-PT-92523:X.Value,1.000000,1.000002,0.999987,1.0,1.000000,1.000001,1.000000,NaN
VAL_23-PDT-92534:X.Value,1.000006,1.000005,0.999925,1.0,1.000000,1.000009,1.000004,NaN
VAL_23_TT_92532:Z.X.Value,1.000000,0.999994,1.000452,1.0,1.000000,0.999988,1.000012,NaN
VAL_23-TIC-92504:Z.X.Value,1.000009,1.000005,0.999976,1.0,1.000000,1.000001,1.000009,NaN
VAL_23_KA_9101_M01_62C:Z.X.Value,0.999999,0.999997,0.999975,1.0,1.000000,1.000000,0.999999,NaN
